<a href="https://colab.research.google.com/github/Sabelz/Master_Thesis_Alexander/blob/main/utils/functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions for the project

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master_Thesis_Alexander
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch > \dev\null # Suppress prints
import gpytorch
from matplotlib import pyplot as plt

Mounted at /content/drive
/content/drive/MyDrive/Master_Thesis_Alexander


# Training Function

In [ ]:
def train(model, likelihood, x_train, y_train, training_iter=10):
    """
    Trains a Gaussian Process (GP) model using the ExactMarginalLogLikelihood loss function.

    Parameters:
    model (gpytorch.models.ExactGP): The GP model to be trained.
    likelihood (gpytorch.likelihoods._OneDimensionalLikelihood): The likelihood function to be used with the model.
    x_train (torch.Tensor): The training data.
    y_train (torch.Tensor): The labels for the training data.
    training_iter (int, optional): The number of training iterations. Default is 10.
    train_loader (torch.utils.data.DataLoader, optional):

    Returns:
    None. The function operates in-place on the `model` and `likelihood` objects.

    Note:
    The function moves the model and likelihood to GPU if available and uses the Adam optimizer for training.
    """
    if torch.cuda.is_available():
      model = model.cuda()
      likelihood = likelihood.cuda()
    model.train()
    likelihood.train()
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(x_train)
        # Calc loss and backprop gradients
        loss = -mll(output, y_train)
        loss.backward()
        optimizer.step()



# Training Function for Inducing Points GP

In [ ]:
def train_ELBO(model, likelihood, x_train, y_train, training_iter=10, train_loader=None):
    """
    Trains a Gaussian Process (GP) model using the VariationalELBO loss function.

    Parameters:
    model (gpytorch.models.ApproximateGP): The GP model to be trained.
    likelihood (gpytorch.likelihoods._OneDimensionalLikelihood): The likelihood function to be used with the model.
    x_train (torch.Tensor): The training data.
    y_train (torch.Tensor): The labels for the training data.
    training_iter (int, optional): The number of training iterations. Default is 10.
    train_loader (torch.utils.data.DataLoader, optional):
    The DataLoader that provides batches of the training data.
    If None, the entire training dataset is used in each iteration.

    Returns:
    None. The function operates in-place on the `model` and `likelihood` objects.

    Note:
    The function moves the model and likelihood to GPU if available and uses the Adam optimizer for training.
    """
    if torch.cuda.is_available():
      model = model.cuda()
      likelihood = likelihood.cuda()
    # Parameters and and input data should be of same dtype
    model = model.double()
    likelihood = likelihood.double()

    model.train()
    likelihood.train()
    # Initialize MLL
    n_points = y_train.numel() # Amount of training points
    # Yes, when training a variational Gaussian Process (GP) model like ApproximateGP,
    # you should use a variational marginal log likelihood (MLL) instead of the exact MLL.
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, n_points) # Loss
    # Use the adam optimizer
    optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

    if(train_loader == None):
      for i in range(training_iter):
          # Zero gradients from previous iteration
          optimizer.zero_grad()
          # Output from model
          output = model(x_train)
          # Calc loss and backprop gradients
          loss = -mll(output, y_train)
          loss.backward()
          optimizer.step()
    else: # If train_loader defined, use it
      for i in range(training_iter):
        for x_batch, y_batch in train_loader:
          # Zero gradients from previous iteration
          optimizer.zero_grad()
          # Output from model
          output = model(x_batch)
          # Calc loss and backprop gradients
          loss = -mll(output, y_batch)
          loss.backward()
          optimizer.step()

# Predict Function

In [ ]:
def predict(model, likelihood, test_x):
    """
    This function makes predictions using a given model and likelihood.

    The function sets the model and likelihood to evaluation mode,
    then computes the likelihood of the model's predictions on the test data.
    It uses PyTorch's `no_grad` context manager to avoid tracking gradients during the prediction,
    and GPyTorch's `fast_pred_var` setting for efficient computation.

    Parameters:
    model (gpytorch.models.GP): The Gaussian Process model to make predictions with.
    likelihood (gpytorch.likelihoods.Likelihood): The likelihood associated with the model.
    test_x (torch.Tensor): The test inputs to make predictions on.

    Returns:
    gpytorch.distributions.MultivariateNormal: The distribution of the model's predictions.
    """
    model.eval()
    likelihood.eval()
    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        return likelihood(model(test_x))

# Plot Function

In [ ]:
def plotGP(x_train, y_train, model, likelihood, title="GP Model"):
  """
    This function plots the Gaussian Process regression model along with the observed data.

    Parameters:
    x_train (torch.Tensor): The training inputs.
    y_train (torch.Tensor): The training targets.
    model (gpytorch.models.GP): The Gaussian Process regression model.
    likelihood (gpytorch.likelihoods.Likelihood): The likelihood function to use for the model.
    title (str, optional): The title of the plot. Defaults to "GP Model".

    Returns:
    None. The function creates a plot and does not return anything.

    """
  # Find min and max value of training set
  min_value, max_value = min(x_train), max(x_train)
  # Create points between min and max values
  x_plot = torch.linspace(min_value, max_value, 1000)
  # Evaluate on plot values
  prediction = likelihood(model(x_plot))
  mean = prediction.mean
  variance = prediction.variance
  with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # Initalize plot
    plt.style.use('dark_background')

    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Confidence region
    lower_bound = mean-(2*(np.sqrt(variance)))
    upper_bound = mean+(2*(np.sqrt(variance)))

    ax.plot(x_train.detach().numpy(), y_train.detach().numpy(), 'wo')
    # Plot predictive means
    ax.plot(x_plot.detach().numpy(), mean.detach().numpy(), 'violet')
    # Plot confidence bounds as lightly shaded region
    ax.fill_between(x_plot.detach().numpy(), lower_bound.detach().numpy(), upper_bound.detach().numpy(), alpha=0.5)
    ax.set_title(title)
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
    plt.grid(False)
    ax.plot
